In [0]:
import bodo
bodo.__version__

In [0]:
import ipyparallel as ipp
ipp.__version__

In [0]:
import os
import sys
import json
from subprocess import run, STDOUT

os.environ['OMPI_ALLOW_RUN_AS_ROOT'] = '1'
os.environ['OMPI_ALLOW_RUN_AS_ROOT_CONFIRM'] = '1'

In [0]:
c = ipp.Cluster(profile="mpi",
                engine_launcher_class='MPI',
                n=n_cores,  # Number of engines, you can change this
                controller_ip='*',
                controller_args=["--nodb"],
                controller_location=spark.conf.get('spark.driver.host')
               ) 
c.engine_launcher_class.mpi_args = ["-machinefile", "/home/ubuntu/.ssh/hostfile"]
c.start_controller_sync()

In [0]:
# Get connection info
connection_info = await c.controller.get_connection_info()
engine_info = connection_info['engine']

In [0]:
os.environ['CONNECTION_INFO'] = json.dumps(engine_info)
os.environ['connection_file'] = os.path.join(
        c.profile_dir,
        'security',
        f'ipcontroller-{c.cluster_id}-engine.json',
    )

In [0]:
# Get connection info
connection_info = await c.controller.get_connection_info()
engine_info = connection_info['engine']


def send_connection_info(connection_info, connection_file):
    env = os.environ.copy()
    env["CONNECTION_INFO"] = json.dumps(connection_info)
    cmd =     [
        'mpiexec',
        '-ppn',
        '1',
        '-machinefile',
        "/home/ubuntu/.ssh/hostfile",
        'sh',
        '-c',
        f'echo $CONNECTION_INFO > "{connection_file}"'

    ]
    p = run(cmd, capture_output=True, text=True, input=None, env=env)
    if p.returncode:
        print(p.stderr, file=sys.stderr)
        p.check_returncode()
    return p

send_connection_info(
    engine_info,
    os.path.join(
        c.profile_dir,
        'security',
        f'ipcontroller-{c.cluster_id}-engine.json',
    ),
)

In [0]:
c.start_engines_sync()
rc = c.connect_client_sync()
rc.wait_for_engines(n=c.n)
view = rc[:]
view.activate()
view.block = True

In [0]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD
print(f"Hello World from rank {comm.Get_rank()} on host {MPI.Get_processor_name()}. total ranks={comm.Get_size()}")